In [1]:
import numpy as np
from scipy.optimize import fsolve
from matplotlib import pyplot as plt

In [2]:
# fixed parameter definitions

beta1 = 1
alpha = -2
gamma0 = 1/2
gamma1 = 1/4
beta2_bar = 4
beta3_bar = 4
sigma2 = 1
sigma3 = 1

# markets and goods
T = 600
J = 4

In [3]:
# 3.1

# x_jt, w_jt are absolute value of iid standard normal variables
x = np.absolute(np.random.standard_normal(size=(J,T)))
w = np.absolute(np.random.standard_normal(size=(J,T)))

unobservable_mean = [0,0]
unobservable_cov = [[1,0.25],[0.25,1]]
unobservables = np.random.multivariate_normal(unobservable_mean, unobservable_cov, size=(J,T))
xi = unobservables[:,:,0]
omega = unobservables[:,:,1]

In [4]:
# 3.2a
# defining the market share

def own_mkt_share_derivative(t, p, beta2, beta3): 
    # p should be a length J vector
    # betas should be num_sims
    
    u_t = np.tile(x[:,t] + xi[:,t] + alpha*p, (len(beta2), 1)) # num_sims x J 
    for j in range(J):
        if j < 2:
            u_t[:,j] = u_t[:,j] + beta2
        else:
            u_t[:,j] = u_t[:,j] + beta3
            
    Z = np.tile( 1 + np.sum(np.exp(u_t),axis=-1), (J,1)).T
    numerator = alpha*np.exp(u_t)*Z - alpha*np.square(np.exp(u_t)) # num_sims x J
    denominator = np.square(Z)
    
    return np.mean(numerator / denominator, axis=0)

    
    
def full_mkt_share_derivative(t, p, beta2, beta3):
    # p should be a length J vector
    # betas should be num_sims
    
    u_t = np.tile(x[:,t] + xi[:,t] + alpha*p, (len(beta2), 1)) # num_sims x J 
    for j in range(J):
        if j < 2:
            u_t[:,j] = u_t[:,j] + beta2
        else:
            u_t[:,j] = u_t[:,j] + beta3
            
    Z = np.tile( 1 + np.sum(np.exp(u_t),axis=-1), (J,1)).T # num_sims x J
    
    derivatives = np.zeros((J,J))
    
    own_numerator = alpha*np.exp(u_t)*Z - alpha*np.square(np.exp(u_t)) # num_sims x J
    denominator = np.square(Z)
    
    for j in range(J):
        derivatives[j,j] = np.mean(own_numerator / denominator, axis=0)[j]
        
    for j in range(J):
        for k in range(J):
            if not (j == k):
                derivatives[j,k] = np.mean(-1*alpha*np.exp(u_t)[:,k]*np.exp(u_t)[:,j] / np.square(1 + np.sum(np.exp(u_t),axis=-1)))
    
    return derivatives

In [5]:
# s_jt(p) 
def mkt_share(t, p, beta2, beta3):
    # p should be a length J vector
    # betas should be num_sims
    
    u_t = np.tile(x[:,t] + xi[:,t] + alpha*p, (len(beta2), 1)) # num_sims x J 
    for j in range(J):
        if j < 2:
            u_t[:,j] = u_t[:,j] + beta2
        else:
            u_t[:,j] = u_t[:,j] + beta3
            
    numerator = np.exp(u_t) 
    denominator = 1 + np.sum(np.exp(u_t),axis=-1) # num_sims
    
    return np.mean(numerator / (np.tile(denominator, (J, 1)).T), axis=0) 


In [6]:
# 3.2a(iv)
# draw beta coefficients for N individuals S times, observe variation in market share derivatives

S = 100

all_derivatives = np.zeros((J,J,S))
all_shares = np.zeros((J,S))

N = 200

price = np.array([1,1,1,1])

for s in range(S):
    beta2 = np.random.normal(beta2_bar, sigma2, N)
    beta3 = np.random.normal(beta3_bar, sigma3, N)
    all_derivatives[:,:,s] = full_mkt_share_derivative(0, price, beta2, beta3)
    all_shares[:,s] = mkt_share(1, price, beta2, beta3)
(np.mean(all_shares,axis=1), np.std(all_shares,axis=1), np.mean(all_derivatives, axis=2), np.std(all_derivatives, axis=2))

(array([0.10762807, 0.1483604 , 0.07428918, 0.65887591]),
 array([0.00625792, 0.00862626, 0.00151823, 0.01346532]),
 array([[-0.3085335 ,  0.0796095 ,  0.13506776,  0.08560505],
        [ 0.0796095 , -0.18877944,  0.0644115 ,  0.04082358],
        [ 0.13506776,  0.0644115 , -0.4297744 ,  0.21854148],
        [ 0.08560505,  0.04082358,  0.21854148, -0.3524195 ]]),
 array([[0.01095348, 0.00637071, 0.00397916, 0.00252197],
        [0.00637071, 0.00817461, 0.0018976 , 0.00120268],
        [0.00397916, 0.0018976 , 0.00679929, 0.00961901],
        [0.00252197, 0.00120268, 0.00961901, 0.00743091]]))

In [7]:
mc = np.exp( gamma0 + gamma1*w + omega/8)

In [16]:
# define function to solve

def get_function_to_solve(t, beta2, beta3):
    def F(p):
        # p is a 
        ds_dp = own_mkt_share_derivative(t, p, beta2, beta3)
        shares = mkt_share(t, p, beta2, beta3)
        return p - mc[:,t] + np.reciprocal(ds_dp)*shares
        
    return F

In [17]:
# draw betas, now compute equilibrium prices and shares

# 3.2 and 3.3: compute equilibrium shares, prices

beta2 = np.random.normal(beta2_bar, sigma2, N)
beta3 = np.random.normal(beta3_bar, sigma3, N)

# these two variables are the prices and shares
eq_prices = np.zeros((J, T))
eq_shares = np.zeros((J, T))

flag_total = 0

for t in range(T):
    fn = get_function_to_solve(t, beta2, beta3)
    mkt_eq_prices, _ , flag, _ = fsolve(fn, np.array([1,1,1,1]), full_output=True)
    flag_total += flag
    eq_prices[:,t] = mkt_eq_prices
    eq_shares[:, t] = mkt_share(t, mkt_eq_prices, beta2, beta3)
    
# this should be True iff all of the fsolves converge
flag_total == T

True

In [21]:
t = 6
eq_shares[:,t], x[:,t], mc[:,t]

(array([0.2580929 , 0.41805876, 0.10362801, 0.05481309]),
 array([1.1733439 , 1.33296936, 1.04494269, 0.54750783]),
 array([1.80080365, 1.89341199, 1.69392408, 2.02867883]))

In [11]:
x.shape

(4, 600)